**Note:** This notebook is intended for Google Colab. Mount Drive or otherwise make your graph files available before running.

# Step 1: Environment Setup

In [1]:
# Step 1: Install and import necessary libraries
# We use !pip to install packages in the Google Colab environment.

!pip install networkx==3.2.1
!pip install matplotlib==3.8.2
!pip install seaborn==0.13.1
!pip install pandas==2.2.1

# Import the libraries for use in our script
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set some default styles for our plots for better aesthetics
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 14

print("All libraries installed and imported successfully.")
print(f"NetworkX version: {nx.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 104.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.1 which is incompatible.
All libraries installed and imported successfully.
NetworkX version: 3.2.1


# Step 2: Mount Google Drive and Load Graph Data

In [2]:
# Step 2: Load the Corrected Graph
import networkx as nx

# The path to our newly corrected file
corrected_path = "/content/drive/MyDrive/knowledge-graph-llms/Backup/graph_corrected.graphml"

try:
    print(f"\nAttempting to load the corrected graph from: {corrected_path}")

    # Load the graph from the corrected file
    G = nx.read_graphml(corrected_path)

    # --- Verification Step ---
    num_nodes = G.number_of_nodes()
    num_edges = G.number_of_edges()

    print("\n--- Graph Summary ---")
    print("SUCCESS! THE GRAPH IS LOADED.")
    print(f"Number of nodes: {num_nodes}")
    print(f"Number of edges: {num_edges}")
    print("---------------------")

except Exception as e:
    print(f"A final error occurred: {e}")


Attempting to load the corrected graph from: /content/drive/MyDrive/knowledge-graph-llms/Backup/graph_corrected.graphml

--- Graph Summary ---
SUCCESS! THE GRAPH IS LOADED.
Number of nodes: 2424
Number of edges: 3241
---------------------


# Step 3: Identify and List All 'Role' Nodes



In [3]:
# Step 3 (Corrected): Identify and List All 'Role' Nodes
import pandas as pd

# We have found the correct attribute key and the format for the value.
ATTRIBUTE_KEY_FOR_NODE_TYPE = 'labels'
ROLE_VALUE = ':Role' # The value includes a colon at the start

# Create a list of all nodes where the type is 'Role'
try:
    role_nodes = [data['id'] for node, data in G.nodes(data=True) if data.get(ATTRIBUTE_KEY_FOR_NODE_TYPE) == ROLE_VALUE]

    # Create a pandas DataFrame for clean display
    roles_df = pd.DataFrame(sorted(role_nodes), columns=['Role Title'])

    print(f"--- Analysis of 'Role' Nodes ---")
    print(f"Total number of unique roles found: {len(roles_df)}")
    print("------------------------------------")

    # Display the full list of roles in a table
    # In Colab, this will be a searchable, sortable table.
    display(roles_df)

except Exception as e:
    print(f"An error occurred: {e}")

--- Analysis of 'Role' Nodes ---
Total number of unique roles found: 171
------------------------------------


,Role Title
0,Administration Assistant
1,Administrative Assistant
2,Analytics And Insights Manager
3,Analytics Developer
4,Apprentice Electricians
...,...
166,Vendor Administrator
167,Warranty Engineer
168,Workforce Participation Manager
169,Workforce Planning Analyst


# Step 4.1: Setup, API Key, and Initial Role Extraction

In [19]:
# Step 4.1: Setup, API Key, and Initial Role Extraction

# --- 1. Install necessary libraries ---
!pip install openai --quiet
!pip install rapidfuzz --quiet
print("Required libraries are installed.")

# --- 2. Import libraries and load API Key ---
import os
import pandas as pd
from google.colab import userdata
from openai import OpenAI

try:
    # Load the API key from Colab secrets
    api_key = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = api_key

    # Instantiate the OpenAI client. It will automatically use the API key from the environment variable.
    # Per your request, we will configure this for text-embedding-3-large later,
    # but instantiating the client now verifies the key is loaded.
    client = OpenAI()

    print("OpenAI API Key loaded and client instantiated successfully.")

except Exception as e:
    print(f"An error occurred while loading the API Key: {e}")
    print("Please ensure your secret is named 'OPENAI_API_KEY' in your Colab notebook.")

# --- 3. Extract Role Nodes from the Graph G ---
# We use the key 'labels' and value ':Role' which we discovered in our earlier inspection.
try:
    all_roles = sorted([
        data['id'] for node, data in G.nodes(data=True)
        if data.get('labels') == ':Role'
    ])

    print(f"\nSuccessfully extracted {len(all_roles)} unique roles from the graph.")

    # Display a sample of the extracted roles
    print("\n--- Sample of first 10 roles ---")
    for role in all_roles[:10]:
        print(role)
    print("--------------------------------")

except NameError:
    print("\nERROR: The graph object 'G' was not found.")
    print("Please ensure you have successfully run the graph loading steps first.")
except Exception as e:
    print(f"\nAn error occurred during role extraction: {e}")

Required libraries are installed.
OpenAI API Key loaded and client instantiated successfully.

Successfully extracted 171 unique roles from the graph.

--- Sample of first 10 roles ---
Administration Assistant
Administrative Assistant
Analytics And Insights Manager
Analytics Developer
Apprentice Electricians
Area Manager
Artificial Intelligence Manager
Asset Engineer
Assistant Ressources Humaines
Automation Manager
--------------------------------


#semantic matching with osca

In [20]:
# --- 1. Install necessary libraries ---
!pip install openai --quiet
!pip install pandas numpy tqdm --quiet

# --- 2. Import libraries and load API Key ---
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from tqdm import tqdm
from google.colab import userdata

# Load OpenAI API key from Colab secrets
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI()

print("OpenAI API Key loaded and client instantiated successfully.")

# --- 3. Get roles ---
# (A) If you want roles from your graph 'G'
try:
    all_roles = sorted([
        data['id'] for node, data in G.nodes(data=True)
        if data.get('labels') == ':Role'
    ])
    print(f"Extracted {len(all_roles)} roles from NetworkX graph G.")
except NameError:
    # (B) Fallback: Load from a CSV as previously
    roles_path = '/content/drive/MyDrive/knowledge-graph-llms/our_roles.csv'
    all_roles = pd.read_csv(roles_path)['Our_Role'].dropna().unique().tolist()
    print(f"Extracted {len(all_roles)} roles from CSV.")

# --- 4. Load OSCA lookup file ---
osca_path = '/content/drive/MyDrive/knowledge-graph-llms/osca_titles_lookup.csv'
osca_df = pd.read_csv(osca_path)
osca_df = osca_df.dropna(subset=['Identifier', 'Description', 'Category'])

# --- 5. Principal Title Lookup ---
principal_titles = (
    osca_df[osca_df['Category'] == 'Principal Title']
    .drop_duplicates(subset=['Identifier'])
    .set_index('Identifier')['Description']
    .to_dict()
)

# --- 6. Generate Embeddings ---
def get_openai_embeddings(text_list, model="text-embedding-3-small"):
    response = client.embeddings.create(
        input=text_list,
        model=model
    )
    return [np.array(e.embedding) for e in response.data]

# Roles embeddings
batch_size = 100
role_embeddings = []
for i in tqdm(range(0, len(all_roles), batch_size), desc="Embedding your roles"):
    batch = all_roles[i:i+batch_size]
    role_embeddings.extend(get_openai_embeddings(batch))

# OSCA descriptions embeddings
osca_descriptions = osca_df['Description'].tolist()
osca_embeddings = []
for i in tqdm(range(0, len(osca_descriptions), batch_size), desc="Embedding OSCA titles"):
    batch = osca_descriptions[i:i+batch_size]
    osca_embeddings.extend(get_openai_embeddings(batch))

# --- 7. Similarity Calculation ---
from numpy.linalg import norm
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

top_k = 3  # Top matches per role

results = []
for i, (role, role_emb) in enumerate(zip(all_roles, role_embeddings)):
    sims = [cosine_similarity(role_emb, osca_emb) for osca_emb in osca_embeddings]
    top_indices = np.argsort(sims)[::-1][:top_k]
    for rank, idx in enumerate(top_indices, 1):
        osca_row = osca_df.iloc[idx]
        principal_title = principal_titles.get(str(osca_row['Identifier']), "")
        results.append({
            'Our_Role': role,
            'OSCA_Identifier': osca_row['Identifier'],
            'OSCA_Description': osca_row['Description'],
            'OSCA_Principal_Title': principal_title,
            'Category': osca_row['Category'],
            'Similarity_Score': round(float(sims[idx]), 4),
            'Rank': rank
        })

# --- 8. Save and Preview ---
results_df = pd.DataFrame(results)
results_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_similarity_matches.csv'
results_df.to_csv(results_path, index=False)
print(f"Saved results to: {results_path}")
display(results_df.head(20))


OpenAI API Key loaded and client instantiated successfully.
Extracted 171 roles from NetworkX graph G.


Embedding OSCA titles: 100%|██████████| 34/34 [00:26<00:00,  1.26it/s]


Saved results to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_similarity_matches.csv


,Our_Role,OSCA_Identifier,OSCA_Description,OSCA_Principal_Title,Category,Similarity_Score,Rank
0,Administration Assistant,599231,HR Administration Assistant,,Alternative Title,0.7845,1
1,Administration Assistant,522231,Executive Assistant,,Principal Title,0.7699,2
2,Administration Assistant,171231,Office Manager,,Principal Title,0.7455,3
3,Administrative Assistant,522231,Executive Assistant,,Principal Title,0.7835,1
4,Administrative Assistant,599231,HR Administration Assistant,,Alternative Title,0.7796,2
5,Administrative Assistant,599231,Human Resources Administration Assistant,,Principal Title,0.7640,3
6,Analytics And Insights Manager,223231,Data Analyst,,Principal Title,0.6272,1
7,Analytics And Insights Manager,113299,ICT Business Intelligence Manager,,Occupation in nec category,0.6257,2
8,Analytics And Insights Manager,221532,Digital Marketing Analyst,,Principal Title,0.5848,3
9,Analytics Developer,273333,Application Developer,,Specialisation,0.7764,1


# Split Matches Into Categories and Save to Files

In [21]:
# Assume results_df is already created as above

# Category 1: Auto-match (exact semantic match)
matched_df = results_df[results_df['Similarity_Score'] == 1.0].copy()
matched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_semantic_matched.csv'
matched_df.to_csv(matched_path, index=False)
print(f"Auto-matched (Similarity=1.0) saved to: {matched_path}")

# Category 2: LLM Review (similarity between 0.8 and 1, exclusive)
llm_review_df = results_df[(results_df['Similarity_Score'] < 1.0) & (results_df['Similarity_Score'] >= 0.8)].copy()
llm_review_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_llm_review.csv'
llm_review_df.to_csv(llm_review_path, index=False)
print(f"LLM review candidates (0.8 ≤ Similarity < 1.0) saved to: {llm_review_path}")

# Category 3: Unmatched (similarity below 0.8)
unmatched_df = results_df[results_df['Similarity_Score'] < 0.8].copy()
unmatched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_unmatched.csv'
unmatched_df.to_csv(unmatched_path, index=False)
print(f"Unmatched roles (Similarity < 0.8) saved to: {unmatched_path}")


Auto-matched (Similarity=1.0) saved to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_semantic_matched.csv
LLM review candidates (0.8 ≤ Similarity < 1.0) saved to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_llm_review.csv
Unmatched roles (Similarity < 0.8) saved to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_unmatched.csv


#LLM Review: One-by-One, With Context from Auto-Matches

In [24]:
# --- LLM Review: Prefer Matching to Existing Auto-Matched OSCA Titles, Otherwise Use Semantic Candidate ---

!pip install openai --quiet

import pandas as pd
import os
import time
from openai import OpenAI
from google.colab import userdata

# Load OpenAI API Key from Colab secrets
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI()

# Load data files
cat1_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_semantic_matched.csv'
llm_review_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_llm_review.csv'

cat1_df = pd.read_csv(cat1_path)
llm_review_df = pd.read_csv(llm_review_path)

# Prepare the list of auto-matched OSCA titles (for the prompt)
auto_matched_list = cat1_df[['OSCA_Description', 'OSCA_Principal_Title']].drop_duplicates().to_dict(orient='records')

# Filter out roles already in category 1 (should not LLM-review those)
auto_matched_roles = set(cat1_df['Our_Role'])
llm_review_df_filtered = llm_review_df[~llm_review_df['Our_Role'].isin(auto_matched_roles)].copy()

# LLM prompt function
def build_llm_prompt(role, candidate, auto_matched_list):
    prompt = (
        "You are an expert in classifying construction workforce roles. "
        "Your first priority is to match the project role to any OSCA title already matched (listed below) if the technical skills and daily responsibilities align—ignore differences in seniority, managerial, or assistant level. "
        "If there is no suitable match among the auto-matched OSCA titles, then assess the semantic similarity candidate OSCA title as a possible match. "
        "Approve a match if the technical skills and job tasks align.\n\n"
        f"Project Role: {role}\n"
        "Auto-matched OSCA Titles:\n"
    )
    for entry in auto_matched_list:
        prompt += f"- {entry['OSCA_Description']} (Principal: {entry['OSCA_Principal_Title']})\n"
    prompt += (
        "\nSemantic Similarity Candidate OSCA Title:\n"
        f"- {candidate['OSCA_Description']} (Principal: {candidate['OSCA_Principal_Title']})\n"
        f"Category: {candidate['Category']}\n"
        f"Similarity Score: {candidate['Similarity_Score']}\n\n"
        "Instructions:\n"
        "1. If the project role matches the technical skills of any auto-matched OSCA title, reply: 'Match: [OSCA title]' and a one-sentence justification.\n"
        "2. If not, consider the semantic similarity candidate OSCA title. If that aligns, reply: 'Match: [Candidate OSCA title]' and a one-sentence justification.\n"
        "3. If neither is a match, reply: 'No suitable match' and a brief justification."
    )
    return prompt

# LLM review loop, one by one
llm_results = []

for idx, row in llm_review_df_filtered.iterrows():
    role = row['Our_Role']
    user_prompt = build_llm_prompt(role, row, auto_matched_list)
    try:
        response = client.chat.completions.create(
            model="o3-mini",
            messages=[
                {"role": "system", "content": "You are an expert in Australian construction workforce roles."},
                {"role": "user", "content": user_prompt}
            ]
        )
        reply = response.choices[0].message.content.strip()
    except Exception as e:
        reply = f"Error: {e}"
    llm_results.append({
        "Our_Role": role,
        "OSCA_Identifier": row['OSCA_Identifier'],
        "OSCA_Description": row['OSCA_Description'],
        "OSCA_Principal_Title": row['OSCA_Principal_Title'],
        "Category": row['Category'],
        "Similarity_Score": row['Similarity_Score'],
        "LLM_Decision": reply
    })
    print(f"Reviewed: {role} | LLM: {reply}")
    time.sleep(1.2)  # Respect OpenAI API rate limits

# Save results
llm_results_df = pd.DataFrame(llm_results)
llm_results_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_llm_reviewed.csv'
llm_results_df.to_csv(llm_results_path, index=False)
print(f"\nLLM-reviewed results saved to: {llm_results_path}")

# Preview
display(llm_results_df.head(10))


Reviewed: Area Manager | LLM: Match: Area Manager (Retail) – The project role’s oversight and management responsibilities align well with those in the candidate title despite the slight industry variation.
Reviewed: Catering Coordinator | LLM: Match: Catering Manager – The role of a Catering Coordinator aligns closely with that of a Catering Manager, as both involve managing catering operations, logistics, and service delivery.
Reviewed: Catering Coordinator | LLM: Match: Catering Assistant – The role involves coordinating catering services, and despite the title difference in seniority, the technical skills and daily responsibilities align closely with those of a catering assistant.
Reviewed: Cloud Infrastructure Engineer | LLM: Match: Cloud Infrastructure Architect – This candidate title aligns closely with the technical skills and daily responsibilities required for designing and managing cloud-based systems inherent in a Cloud Infrastructure Engineer role.
Reviewed: Cloud Infrastru

,Our_Role,OSCA_Identifier,OSCA_Description,OSCA_Principal_Title,Category,Similarity_Score,LLM_Decision
0,Area Manager,149999,Area Manager (Retail),NaN,Occupation in nec category,0.8695,Match: Area Manager (Retail) – The project rol...
1,Catering Coordinator,161232,Catering Manager,NaN,Principal Title,0.8629,Match: Catering Manager – The role of a Cateri...
2,Catering Coordinator,851231,Catering Assistant,NaN,Specialisation,0.8011,Match: Catering Assistant – The role involves ...
3,Cloud Infrastructure Engineer,273231,Cloud Infrastructure Architect,NaN,Alternative Title,0.9213,Match: Cloud Infrastructure Architect – This c...
4,Cloud Infrastructure Engineer,273331,Cloud Engineer,NaN,Principal Title,0.8543,Match: Cloud Engineer – The role of a Cloud In...
5,Contracts Administrator,511131,Contract Administrator,NaN,Principal Title,0.9274,Match: Contract Administrator — The candidate ...
6,Control Systems Engineer,243533,Control Systems Engineer (Production or Plant),NaN,Specialisation,0.8139,Match: Control Systems Engineer (Production or...
7,Data Engineering Manager,223233,Data Engineer,NaN,Principal Title,0.8403,Match: Data Engineer. The technical expertise ...
8,Data Engineers,223233,Data Engineer,NaN,Principal Title,0.8949,"Match: Data Engineer \nThe project role ""Data..."
9,Data Support Analyst,223231,Data Analyst,NaN,Principal Title,0.8091,Match: Data Analyst \nThe role of a Data Supp...


In [25]:
import pandas as pd

# Load previous outputs
auto_matched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_semantic_matched.csv'
llm_reviewed_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_llm_reviewed.csv'
semantic_unmatched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/osca_semantic_unmatched.csv'

auto_matched_df = pd.read_csv(auto_matched_path)
llm_reviewed_df = pd.read_csv(llm_reviewed_path)
semantic_unmatched_df = pd.read_csv(semantic_unmatched_path)

# 1. All matches (for graph update)
llm_matched_df = llm_reviewed_df[llm_reviewed_df['LLM_Decision'].str.lower().str.startswith('match')].copy()
all_matched_df = pd.concat([auto_matched_df, llm_matched_df], ignore_index=True)

all_matched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_all_matched.csv'
all_matched_df.to_csv(all_matched_path, index=False)
print(f"All matched (semantic + LLM) saved to: {all_matched_path}")

# 2. Final unmatched
llm_unmatched_df = llm_reviewed_df[llm_reviewed_df['LLM_Decision'].str.lower().str.startswith('no suitable match')].copy()
final_unmatched_df = pd.concat([semantic_unmatched_df, llm_unmatched_df], ignore_index=True)

final_unmatched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/final_unmatched_roles.csv'
final_unmatched_df.to_csv(final_unmatched_path, index=False)
print(f"Final unmatched roles saved to: {final_unmatched_path}")

# Preview outputs
print("\n--- Preview: All matched roles ---")
display(all_matched_df.head(10))
print("\n--- Preview: Final unmatched roles ---")
display(final_unmatched_df.head(10))


All matched (semantic + LLM) saved to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_all_matched.csv
Final unmatched roles saved to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/final_unmatched_roles.csv

--- Preview: All matched roles ---


,Our_Role,OSCA_Identifier,OSCA_Description,OSCA_Principal_Title,Category,Similarity_Score,Rank,LLM_Decision
0,Automation Technician,313999,Automation Technician,NaN,Occupation in nec category,1.0,1.0,NaN
1,Communications Technician,382334,Communications Technician,NaN,Alternative Title,1.0,1.0,NaN
2,Data Architect,223232,Data Architect,NaN,Principal Title,1.0,1.0,NaN
3,Data Engineer,223233,Data Engineer,NaN,Principal Title,1.0,1.0,NaN
4,Data Scientist,223234,Data Scientist,NaN,Principal Title,1.0,1.0,NaN
5,Digital Marketing Analyst,221532,Digital Marketing Analyst,NaN,Principal Title,1.0,1.0,NaN
6,Electrical Engineer,243331,Electrical Engineer,NaN,Principal Title,1.0,1.0,NaN
7,Electrical Engineering Technician,313232,Electrical Engineering Technician,NaN,Principal Title,1.0,1.0,NaN
8,Electrical Fitter,381232,Electrical Fitter,NaN,Principal Title,1.0,1.0,NaN
9,Environmental Officer,244332,Environmental Officer,NaN,Alternative Title,1.0,1.0,NaN



--- Preview: Final unmatched roles ---


,Our_Role,OSCA_Identifier,OSCA_Description,OSCA_Principal_Title,Category,Similarity_Score,Rank,LLM_Decision
0,Administration Assistant,599231,HR Administration Assistant,NaN,Alternative Title,0.7845,1.0,NaN
1,Administration Assistant,522231,Executive Assistant,NaN,Principal Title,0.7699,2.0,NaN
2,Administration Assistant,171231,Office Manager,NaN,Principal Title,0.7455,3.0,NaN
3,Administrative Assistant,522231,Executive Assistant,NaN,Principal Title,0.7835,1.0,NaN
4,Administrative Assistant,599231,HR Administration Assistant,NaN,Alternative Title,0.7796,2.0,NaN
5,Administrative Assistant,599231,Human Resources Administration Assistant,NaN,Principal Title,0.7640,3.0,NaN
6,Analytics And Insights Manager,223231,Data Analyst,NaN,Principal Title,0.6272,1.0,NaN
7,Analytics And Insights Manager,113299,ICT Business Intelligence Manager,NaN,Occupation in nec category,0.6257,2.0,NaN
8,Analytics And Insights Manager,221532,Digital Marketing Analyst,NaN,Principal Title,0.5848,3.0,NaN
9,Analytics Developer,273333,Application Developer,NaN,Specialisation,0.7764,1.0,NaN


In [28]:
import pandas as pd

# Load your matched file
matched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_all_matched.csv'
df = pd.read_csv(matched_path)

def select_best_per_role(subdf):
    principals = subdf[subdf['Category'].str.lower() == 'principal title']
    if not principals.empty:
        # Pick the Principal Title with highest similarity
        return principals.loc[principals['Similarity_Score'].idxmax()]
    else:
        # No principal: pick the highest similarity among others
        return subdf.loc[subdf['Similarity_Score'].idxmax()]

best_df = df.groupby('Our_Role', group_keys=False).apply(select_best_per_role).reset_index(drop=True)

# Save output
output_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_one_to_one_matched.csv'
best_df.to_csv(output_path, index=False)
print(f"Strict 1-to-1 matched (principal-priority) roles saved to: {output_path}")

# Preview
display(best_df.head(20))


Strict 1-to-1 matched (principal-priority) roles saved to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_one_to_one_matched.csv


/tmp/ipython-input-28-849336326.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_df = df.groupby('Our_Role', group_keys=False).apply(select_best_per_role).reset_index(drop=True)


,Our_Role,OSCA_Identifier,OSCA_Description,OSCA_Principal_Title,Category,Similarity_Score,Rank,LLM_Decision
0,Area Manager,149999,Area Manager (Retail),NaN,Occupation in nec category,0.8695,NaN,Match: Area Manager (Retail) – The project rol...
1,Automation Technician,313999,Automation Technician,NaN,Occupation in nec category,1.0000,1.0,NaN
2,Catering Coordinator,161232,Catering Manager,NaN,Principal Title,0.8629,NaN,Match: Catering Manager – The role of a Cateri...
3,Cloud Infrastructure Engineer,273331,Cloud Engineer,NaN,Principal Title,0.8543,NaN,Match: Cloud Engineer – The role of a Cloud In...
4,Communications Technician,382334,Communications Technician,NaN,Alternative Title,1.0000,1.0,NaN
5,Contracts Administrator,511131,Contract Administrator,NaN,Principal Title,0.9274,NaN,Match: Contract Administrator — The candidate ...
6,Control Systems Engineer,243533,Control Systems Engineer (Production or Plant),NaN,Specialisation,0.8139,NaN,Match: Control Systems Engineer (Production or...
7,Data Architect,223232,Data Architect,NaN,Principal Title,1.0000,1.0,NaN
8,Data Engineer,223233,Data Engineer,NaN,Principal Title,1.0000,1.0,NaN
9,Data Engineering Manager,223233,Data Engineer,NaN,Principal Title,0.8403,NaN,Match: Data Engineer. The technical expertise ...


In [30]:
# --- 1. Install only pandas (if not present) ---
!pip install pandas --quiet

import pandas as pd

# --- 2. Extract roles from the graph G ---
try:
    all_roles = sorted([
        data['id'] for node, data in G.nodes(data=True)
        if data.get('labels') == ':Role'
    ])
    print(f"\nSuccessfully extracted {len(all_roles)} unique roles from the graph.")
    print("--- Sample of first 10 roles ---")
    for role in all_roles[:10]:
        print(role)
    print("--------------------------------")
except NameError:
    print("\nERROR: The graph object 'G' was not found.")
    all_roles = []
except Exception as e:
    print(f"\nAn error occurred during role extraction: {e}")
    all_roles = []

# --- 3. Compare with matched roles and report ---
matched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_one_to_one_matched.csv'
matched_df = pd.read_csv(matched_path)

all_roles_set = set(all_roles)
matched_roles_set = set(matched_df['Our_Role'].dropna().unique())
unmatched_roles = sorted(all_roles_set - matched_roles_set)

# Reporting
total_roles = len(all_roles_set)
matched_count = len(matched_roles_set & all_roles_set)
unmatched_count = len(unmatched_roles)
matched_percent = matched_count / total_roles * 100 if total_roles else 0
unmatched_percent = unmatched_count / total_roles * 100 if total_roles else 0

print(f"\n=== Matching Summary ===")
print(f"Total roles extracted from graph: {total_roles}")
print(f"Roles with OSCA match:           {matched_count} ({matched_percent:.1f}%)")
print(f"Roles without a match:           {unmatched_count} ({unmatched_percent:.1f}%)")

# --- 4. Save unmatched roles for further processing ---
unmatched_roles_df = pd.DataFrame({'Our_Role': unmatched_roles})
unmatched_save_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/unmatched_roles_for_review.csv'
unmatched_roles_df.to_csv(unmatched_save_path, index=False)
print(f"Unmatched roles saved to: {unmatched_save_path}")

# Preview
display(unmatched_roles_df.head(10))



Successfully extracted 171 unique roles from the graph.
--- Sample of first 10 roles ---
Administration Assistant
Administrative Assistant
Analytics And Insights Manager
Analytics Developer
Apprentice Electricians
Area Manager
Artificial Intelligence Manager
Asset Engineer
Assistant Ressources Humaines
Automation Manager
--------------------------------

=== Matching Summary ===
Total roles extracted from graph: 171
Roles with OSCA match:           53 (31.0%)
Roles without a match:           118 (69.0%)
Unmatched roles saved to: /content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/unmatched_roles_for_review.csv


,Our_Role
0,Administration Assistant
1,Administrative Assistant
2,Analytics And Insights Manager
3,Analytics Developer
4,Apprentice Electricians
5,Artificial Intelligence Manager
6,Asset Engineer
7,Assistant Ressources Humaines
8,Automation Manager
9,Automation Paralegal


In [3]:
# Import networkx if not already imported
import networkx as nx

# Basic graph statistics
print(f"Total nodes in graph:        {G.number_of_nodes()}")
print(f"Total relationships (edges): {G.number_of_edges()}")

# Node type counts (by 'labels')
from collections import Counter

node_labels = [data.get('labels', 'UNKNOWN') for _, data in G.nodes(data=True)]
label_counts = Counter(node_labels)
print("\nNode counts by type (labels):")
for label, count in label_counts.items():
    print(f"  {label}: {count}")

# Edge type counts (by 'type', if present)
if G.number_of_edges() > 0:
    edge_types = [data.get('type', 'UNKNOWN') for _, _, data in G.edges(data=True)]
    edge_type_counts = Counter(edge_types)
    print("\nRelationship counts by type (edge 'type'):")
    for etype, count in edge_type_counts.items():
        print(f"  {etype}: {count}")
else:
    print("\nNo relationships (edges) present in graph.")

# Optional: show a few sample nodes and edges
print("\n--- Sample nodes ---")
for n, d in list(G.nodes(data=True))[:5]:
    print(f"{n}: {d}")
print("\n--- Sample edges ---")
for s, t, d in list(G.edges(data=True))[:5]:
    print(f"{s} -> {t}: {d}")


Total nodes in graph:        2424
Total relationships (edges): 3241

Node counts by type (labels):
  :Role: 171
  :Technical skill: 912
  :Soft skill: 198
  :Task: 824
  :Tool: 317
  :Skill: 2

Relationship counts by type (edge 'type'):
  UNKNOWN: 3241

--- Sample nodes ---
n0: {'labels': ':Role', 'id': 'Analytics And Insights Manager'}
n1: {'labels': ':Technical skill', 'id': 'Predictive Analytics'}
n2: {'labels': ':Technical skill', 'id': 'Machine Learning'}
n3: {'labels': ':Soft skill', 'id': 'Communication'}
n4: {'labels': ':Soft skill', 'id': 'Project Management'}

--- Sample edges ---
n0 -> n1: {'label': 'REQUIRES_SKILL'}
n0 -> n2: {'label': 'REQUIRES_SKILL'}
n0 -> n3: {'label': 'REQUIRES_SKILL'}
n0 -> n4: {'label': 'REQUIRES_SKILL'}
n0 -> n12: {'label': 'USES_TOOL'}


# Step 0: Create a Graph Backup Before Any Change



In [11]:
import networkx as nx
import os

backup_dir = '/content/drive/MyDrive/knowledge-graph-llms/graph_backups'
os.makedirs(backup_dir, exist_ok=True)
backup_path = os.path.join(backup_dir, 'G_before_osca_update.graphml')
nx.write_graphml(G, backup_path)
print(f"Backup saved to: {backup_path}")


Backup saved to: /content/drive/MyDrive/knowledge-graph-llms/graph_backups/G_before_osca_update.graphml


#Step 1: Prepare Lookup Index for Role Nodes (Performance Improvement)

In [12]:
# Build fast lookup: role name -> node key
role_name_to_node_key = {
    d.get('id'): n
    for n, d in G.nodes(data=True)
    if d.get('labels') == ':Role' and d.get('id') is not None
}
print(f"Role node index created. Unique roles in graph: {len(role_name_to_node_key)}")


Role node index created. Unique roles in graph: 171


#Step 2: Add/Update OSCAOccupation Nodes and Enriched Relationships

In [13]:
import pandas as pd

osca_csv = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_one_to_one_matched.csv'
osca_df = pd.read_csv(osca_csv)

added_osca_nodes = 0
added_edges = 0
skipped_roles = 0

for _, row in osca_df.iterrows():
    osca_id = str(row['OSCA_Identifier'])
    osca_node_key = f"osca_{osca_id}"
    # Create or update OSCAOccupation node
    if not G.has_node(osca_node_key):
        G.add_node(
            osca_node_key,
            labels=':OSCAOccupation',
            osca_id=osca_id,
            description=row['OSCA_Description'],
            principal_title=row.get('OSCA_Principal_Title', ''),
            category=row.get('Category', '')
        )
        added_osca_nodes += 1

    # Fast lookup for Role node
    our_role_name = row['Our_Role']
    our_role_node = role_name_to_node_key.get(our_role_name)
    if not our_role_node:
        print(f"[Warning] Role node '{our_role_name}' not found in graph. Skipping.")
        skipped_roles += 1
        continue

    # Add enriched edge (with match metadata)
    G.add_edge(
        our_role_node,
        osca_node_key,
        label='MATCHED_TO_OSCA',
        similarity_score=row.get('Similarity_Score', ''),
        llm_decision=row.get('LLM_Decision', ''),
        rank=row.get('Rank', '')
    )
    added_edges += 1

print(f"\nAdded {added_osca_nodes} OSCAOccupation nodes (if not already present).")
print(f"Created {added_edges} MATCHED_TO_OSCA relationships.")
if skipped_roles:
    print(f"Skipped {skipped_roles} unmatched roles (not present in graph).")



Added 43 OSCAOccupation nodes (if not already present).
Created 53 MATCHED_TO_OSCA relationships.


In [14]:
# List all OSCAOccupation nodes with key properties
osca_nodes = [
    (n, d.get('osca_id'), d.get('description'), d.get('principal_title'), d.get('category'))
    for n, d in G.nodes(data=True)
    if d.get('labels') == ':OSCAOccupation'
]

osca_nodes_df = pd.DataFrame(
    osca_nodes,
    columns=['Node Key', 'OSCA_Identifier', 'OSCA_Description', 'OSCA_Principal_Title', 'Category']
)

print(f"Total OSCAOccupation nodes in graph: {len(osca_nodes_df)}")
display(osca_nodes_df.head(20))


Total OSCAOccupation nodes in graph: 43


,Node Key,OSCA_Identifier,OSCA_Description,OSCA_Principal_Title,Category
0,osca_149999,149999,Area Manager (Retail),NaN,Occupation in nec category
1,osca_313999,313999,Automation Technician,NaN,Occupation in nec category
2,osca_161232,161232,Catering Manager,NaN,Principal Title
3,osca_273331,273331,Cloud Engineer,NaN,Principal Title
4,osca_382334,382334,Communications Technician,NaN,Alternative Title
5,osca_511131,511131,Contract Administrator,NaN,Principal Title
6,osca_243533,243533,Control Systems Engineer (Production or Plant),NaN,Specialisation
7,osca_223232,223232,Data Architect,NaN,Principal Title
8,osca_223233,223233,Data Engineer,NaN,Principal Title
9,osca_223234,223234,Data Scientist,NaN,Principal Title


# Methodologically sound, stepwise LLM-powered pipeline for noise detection

In [ ]:
# --- 1. Install OpenAI library (quietly) ---
!pip install openai --quiet

Cell 2: Load Unmatched Roles and API Key

In [18]:
# --- 2. Load unmatched roles and OpenAI API key ---
import pandas as pd
import os
from google.colab import userdata
from openai import OpenAI

# Load API key
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key
client = OpenAI()

# Load unmatched roles list
unmatched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/unmatched_roles_for_review.csv'
unmatched_df = pd.read_csv(unmatched_path)
roles_to_check = unmatched_df['Our_Role'].dropna().unique()

print(f"Total roles to review: {len(roles_to_check)}")
display(unmatched_df.head(10))


Total roles to review: 118


,Our_Role
0,Administration Assistant
1,Administrative Assistant
2,Analytics And Insights Manager
3,Analytics Developer
4,Apprentice Electricians
5,Artificial Intelligence Manager
6,Asset Engineer
7,Assistant Ressources Humaines
8,Automation Manager
9,Automation Paralegal


Cell 3: Define Refined LLM Noise Check Prompt

In [19]:
# --- 3. Define LLM prompt for noise detection (English, knowledge work only) ---
def build_noise_check_prompt(role_name):
    return (
        "You are an expert in construction workforce and occupation classification in Australia. "
        f"Review the following entry: '{role_name}'.\n\n"
        "Does this represent a real job, profession, or legitimate occupational role related to knowledge work "
        "(rather than physical or site-based manual work) in the construction or built environment sector, and is the role title in English? "
        "If yes, reply 'Valid'. "
        "If not, reply 'Noise' and briefly explain why (e.g., not knowledge work, not English, placeholder, academic title, software, irrelevant to construction, etc.)."
    )


Cell 4: Run LLM Review on Each Unmatched Role

In [20]:
# --- 4. Review each unmatched role with LLM for noise detection ---
import time

results = []
for role in roles_to_check:
    user_prompt = build_noise_check_prompt(role)
    try:
        response = client.chat.completions.create(
            model="o3-mini",
            messages=[
                {"role": "system", "content": "You are an expert in the Australian construction workforce and occupation taxonomy."},
                {"role": "user", "content": user_prompt}
            ]
        )
        reply = response.choices[0].message.content.strip()
    except Exception as e:
        reply = f"Error: {e}"
    results.append({
        "Our_Role": role,
        "LLM_Review": reply
    })
    print(f"Reviewed: {role} | LLM: {reply}")
    time.sleep(1.2)  # Respect API rate limits

results_df = pd.DataFrame(results)
output_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/unmatched_roles_llm_noise_check.csv'
results_df.to_csv(output_path, index=False)
print(f"\nNoise-checked results saved to: {output_path}")

display(results_df.head(20))


Reviewed: Administration Assistant | LLM: Valid
Reviewed: Administrative Assistant | LLM: Valid
Reviewed: Analytics And Insights Manager | LLM: Valid
Reviewed: Analytics Developer | LLM: Valid
Reviewed: Apprentice Electricians | LLM: Noise – While Apprentice Electricians is a legitimate and real occupational role in the construction sector, it is primarily a manual, site‐based trade rather than a role focused on knowledge work.
Reviewed: Artificial Intelligence Manager | LLM: Valid. The title is in English and implies a knowledge-based role that could logically exist within the construction or built environment sector, particularly as digital and AI technologies become more integrated into management and operational processes in the industry.
Reviewed: Asset Engineer | LLM: Valid
Reviewed: Assistant Ressources Humaines | LLM: Noise – The title is in French ("Assistant Ressources Humaines"), not English, even though it denotes a knowledge work role in Human Resources.
Reviewed: Automati

,Our_Role,LLM_Review
0,Administration Assistant,Valid
1,Administrative Assistant,Valid
2,Analytics And Insights Manager,Valid
3,Analytics Developer,Valid
4,Apprentice Electricians,Noise – While Apprentice Electricians is a leg...
5,Artificial Intelligence Manager,Valid. The title is in English and implies a k...
6,Asset Engineer,Valid
7,Assistant Ressources Humaines,"Noise – The title is in French (""Assistant Res..."
8,Automation Manager,Valid
9,Automation Paralegal,"Noise – The title ""Automation Paralegal"" does ..."


# Deleting “Noise” roles from your graph

Cell 1: Isolate the “Noise” Roles List for Deletion

In [22]:
# --- 1. Isolate Noise Roles for Deletion (Robust Encoding) ---
import pandas as pd

noise_file = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/unmatched_roles_llm_noise_check.csv'

# Try UTF-8 first, then fall back to latin1 if needed
try:
    noise_df = pd.read_csv(noise_file, encoding='utf-8-sig')
except UnicodeDecodeError:
    noise_df = pd.read_csv(noise_file, encoding='latin1')

# Extract only roles where LLM_Review begins with "Noise"
noise_roles = noise_df[noise_df['LLM_Review'].str.lower().str.startswith('noise')]['Our_Role'].dropna().unique().tolist()

print(f"Roles identified as 'Noise': {len(noise_roles)}")
print("Sample noise roles:", noise_roles[:10])
# --- Checkpoint: Review the list above. Approve before proceeding. ---


Roles identified as 'Noise': 38
Sample noise roles: ['Apprentice Electricians', 'Assistant Ressources Humaines', 'Automation Paralegal', 'Cadet', 'Comptable', 'Conducteur De Travaux', 'Data Experts', 'Dessinateur Bureau Detudes', 'Digital Intern', 'Electrician Assistant']


Cell 2: Dry Run — Check Existence and Relationships for Each Target Role


In [23]:
# --- 2. Dry Run: Check Existence and Relationships for Target Roles ---
print("Dry run — Reviewing nodes and their relationships (no deletions will be made):\n")

from collections import defaultdict

found_nodes = []
not_found_nodes = []
node_rel_summary = defaultdict(list)

for role_name in noise_roles:
    role_nodes = [
        n for n, d in G.nodes(data=True)
        if d.get('labels') == ':Role' and d.get('id') == role_name
    ]
    if not role_nodes:
        not_found_nodes.append(role_name)
        continue
    found_nodes.append(role_name)
    node = role_nodes[0]
    rels = list(G.edges(node, data=True)) + list(G.in_edges(node, data=True))
    node_rel_summary[role_name] = rels
    print(f"Role '{role_name}' found. Total relationships: {len(rels)}")
    for src, tgt, data in rels:
        print(f"  {src} -> {tgt} | {data}")

print(f"\nTotal roles found for deletion: {len(found_nodes)}")
print(f"Roles NOT found in graph: {not_found_nodes}")
# --- Checkpoint: Review the printout to confirm these are the intended nodes for deletion. ---


Dry run — Reviewing nodes and their relationships (no deletions will be made):

Role 'Apprentice Electricians' found. Total relationships: 5
  n736 -> n738 | {'label': 'REQUIRES_SKILL'}
  n736 -> n739 | {'label': 'REQUIRES_SKILL'}
  n736 -> n740 | {'label': 'REQUIRES_SKILL'}
  n736 -> n741 | {'label': 'REQUIRES_SKILL'}
  n736 -> n737 | {'label': 'PERFORMS_TASK'}
Role 'Assistant Ressources Humaines' found. Total relationships: 7
  n1864 -> n3 | {'label': 'REQUIRES_SKILL'}
  n1864 -> n1869 | {'label': 'REQUIRES_SKILL'}
  n1864 -> n1870 | {'label': 'REQUIRES_SKILL'}
  n1864 -> n1871 | {'label': 'REQUIRES_SKILL'}
  n1864 -> n1865 | {'label': 'PERFORMS_TASK'}
  n1864 -> n1866 | {'label': 'PERFORMS_TASK'}
  n1864 -> n1867 | {'label': 'PERFORMS_TASK'}
Role 'Automation Paralegal' found. Total relationships: 6
  n1321 -> n74 | {'label': 'REQUIRES_SKILL'}
  n1321 -> n1328 | {'label': 'REQUIRES_SKILL'}
  n1321 -> n443 | {'label': 'USES_TOOL'}
  n1321 -> n1329 | {'label': 'USES_TOOL'}
  n1321 -> n

In [24]:
# --- Search for variants of 'Monteur En Sécurité Incendie' ---
suspect_fragment = "Monteur En"
possible_matches = [
    (n, d.get('id')) for n, d in G.nodes(data=True)
    if d.get('labels') == ':Role' and d.get('id') and suspect_fragment.lower() in d.get('id').lower()
]

print("Possible graph nodes matching or resembling 'Monteur En Sécurité Incendie':")
for node_key, node_id in possible_matches:
    print(f"Graph node: {node_key}, id: '{node_id}'")


Possible graph nodes matching or resembling 'Monteur En Sécurité Incendie':
Graph node: n1803, id: 'Monteur En Sécurité Incendie'


In [25]:
# --- Patch noise_roles list for proper encoding ---
# Remove garbled entry if present
noise_roles = [r for r in noise_roles if 'monteur en' not in r.lower()]
# Add correct version
noise_roles.append('Monteur En Sécurité Incendie')
print("'Monteur En Sécurité Incendie' added to deletion list.")


'Monteur En Sécurité Incendie' added to deletion list.


Cell 3: Perform Safe DETACH DELETE of Noise Roles from Graph

In [26]:
# --- 3. Delete Noise Roles from Graph (with DETACH logic) ---
print("Deleting confirmed 'Noise' roles from the graph...")

deleted = 0
for role_name in noise_roles:
    role_nodes = [
        n for n, d in G.nodes(data=True)
        if d.get('labels') == ':Role' and d.get('id') == role_name
    ]
    if not role_nodes:
        continue
    node = role_nodes[0]
    # Remove all edges (DETACH logic)
    G.remove_edges_from(list(G.out_edges(node)))
    G.remove_edges_from(list(G.in_edges(node)))
    # Now remove the node itself
    G.remove_node(node)
    print(f"Deleted node '{role_name}' and its relationships.")
    deleted += 1

print(f"\nTotal roles deleted: {deleted}")


Deleting confirmed 'Noise' roles from the graph...
Deleted node 'Apprentice Electricians' and its relationships.
Deleted node 'Assistant Ressources Humaines' and its relationships.
Deleted node 'Automation Paralegal' and its relationships.
Deleted node 'Cadet' and its relationships.
Deleted node 'Comptable' and its relationships.
Deleted node 'Conducteur De Travaux' and its relationships.
Deleted node 'Data Experts' and its relationships.
Deleted node 'Dessinateur Bureau Detudes' and its relationships.
Deleted node 'Digital Intern' and its relationships.
Deleted node 'Electrician Assistant' and its relationships.
Deleted node 'Electrician Team Leader' and its relationships.
Deleted node 'Graduate Program' and its relationships.
Deleted node 'Graphic Design Intern' and its relationships.
Deleted node 'House Coordinator' and its relationships.
Deleted node 'Monteur' and its relationships.
Deleted node 'Painters/Blasters' and its relationships.
Deleted node 'People Projects Partner' and i

In [28]:
import pickle

with open('/content/drive/MyDrive/knowledge-graph-llms/cleaned_graph_after_noise_removal.gpickle', 'wb') as f:
    pickle.dump(G, f)

print("Cleaned graph saved as pickle.")


Cleaned graph saved as pickle.


In [29]:
nx.write_graphml(G, '/content/drive/MyDrive/knowledge-graph-llms/cleaned_graph_after_noise_removal.graphml')
print("Cleaned graph saved as GraphML.")


Cleaned graph saved as GraphML.


Summary Statistics for the Cleaned Graph

In [30]:
# --- Overview of Cleaned Graph ---
from collections import Counter

node_labels = [d.get('labels', 'UNKNOWN') for _, d in G.nodes(data=True)]
node_label_counts = Counter(node_labels)

edge_labels = [d.get('label', 'UNKNOWN') for _, _, d in G.edges(data=True)]
edge_label_counts = Counter(edge_labels)

print(f"Total nodes:        {G.number_of_nodes()}")
print(f"Total relationships: {G.number_of_edges()}\n")

print("Node counts by type:")
for label, count in node_label_counts.items():
    print(f"  {label}: {count}")

print("\nRelationship counts by type:")
for label, count in edge_label_counts.items():
    print(f"  {label}: {count}")

# Preview a few example nodes and edges
print("\n--- Sample nodes ---")
for n, d in list(G.nodes(data=True))[:5]:
    print(f"{n}: {d}")

print("\n--- Sample edges ---")
for u, v, d in list(G.edges(data=True))[:5]:
    print(f"{u} -> {v}: {d}")


Total nodes:        2429
Total relationships: 2883

Node counts by type:
  :Role: 133
  :Technical skill: 912
  :Soft skill: 198
  :Task: 824
  :Tool: 317
  :Skill: 2
  :OSCAOccupation: 43

Relationship counts by type:
  REQUIRES_SKILL: 1029
  USES_TOOL: 284
  PERFORMS_TASK: 749
  INVOLVES_SKILL: 489
  REQUIRES_TOOL: 246
  MATCHED_TO_OSCA: 53
  REQUIRES_EXPERIENCE: 33

--- Sample nodes ---
n0: {'labels': ':Role', 'id': 'Analytics And Insights Manager'}
n1: {'labels': ':Technical skill', 'id': 'Predictive Analytics'}
n2: {'labels': ':Technical skill', 'id': 'Machine Learning'}
n3: {'labels': ':Soft skill', 'id': 'Communication'}
n4: {'labels': ':Soft skill', 'id': 'Project Management'}

--- Sample edges ---
n0 -> n1: {'label': 'REQUIRES_SKILL'}
n0 -> n2: {'label': 'REQUIRES_SKILL'}
n0 -> n3: {'label': 'REQUIRES_SKILL'}
n0 -> n4: {'label': 'REQUIRES_SKILL'}
n0 -> n12: {'label': 'USES_TOOL'}


In [31]:
# --- Save a GraphML Backup in Your Backup Folder ---
import networkx as nx

backup_path = '/content/drive/MyDrive/knowledge-graph-llms/graph_backups/cleaned_graph_after_noise_removal.graphml'
nx.write_graphml(G, backup_path)
print(f"Backup of cleaned graph saved to: {backup_path}")


Backup of cleaned graph saved to: /content/drive/MyDrive/knowledge-graph-llms/graph_backups/cleaned_graph_after_noise_removal.graphml


In [32]:
role_titles = sorted([d.get('id') for _, d in G.nodes(data=True) if d.get('labels') == ':Role'])
print(f"Total unique roles to review: {len(role_titles)}")
# Optionally, save for manual/LLM review
import pandas as pd
roles_df = pd.DataFrame({'Role': role_titles})
roles_path = '/content/drive/MyDrive/knowledge-graph-llms/role_titles_for_llm_consolidation.csv'
roles_df.to_csv(roles_path, index=False)
print(f"Role list saved for LLM review: {roles_path}")


Total unique roles to review: 133
Role list saved for LLM review: /content/drive/MyDrive/knowledge-graph-llms/role_titles_for_llm_consolidation.csv


In [33]:
import pandas as pd

unmatched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/unmatched_roles_for_review.csv'
unmatched_df = pd.read_csv(unmatched_path)
unmatched_roles = sorted(unmatched_df['Our_Role'].dropna().unique())

print(f"Total unmatched roles: {len(unmatched_roles)}")

# Save for LLM review
roles_unmatched_path = '/content/drive/MyDrive/knowledge-graph-llms/unmatched_role_titles_for_llm_consolidation.csv'
pd.DataFrame({'Role': unmatched_roles}).to_csv(roles_unmatched_path, index=False)
print(f"Unmatched roles saved for LLM review: {roles_unmatched_path}")


Total unmatched roles: 118
Unmatched roles saved for LLM review: /content/drive/MyDrive/knowledge-graph-llms/unmatched_role_titles_for_llm_consolidation.csv


# Self-contained Python script for semantically driven, seniority-agnostic role consolidation

In [36]:
# =============================
# 1. Load Cleaned Graph & Matched Roles
# =============================
import networkx as nx
import pandas as pd

# --- Load cleaned graph ---
graph_path = '/content/drive/MyDrive/knowledge-graph-llms/graph_backups/cleaned_graph_after_noise_removal.graphml'
G = nx.read_graphml(graph_path)
print(f"Graph loaded. Total nodes: {G.number_of_nodes()}")

# --- Load matched roles from OSCA mapping ---
matched_path = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/for_graph-update/osca_one_to_one_matched.csv'
matched = pd.read_csv(matched_path)
matched_roles = set(matched['Our_Role'].dropna())

# --- Extract roles (nodes with label ':Role') from graph ---
graph_roles = set(d['id'] for _, d in G.nodes(data=True) if d.get('labels') == ':Role')

# --- Compute unmatched roles ---
unmatched_roles = sorted(graph_roles - matched_roles)

print(f"Total unique :Role nodes in graph: {len(graph_roles)}")
print(f"Total roles matched with OSCA:     {len(matched_roles)}")
print(f"Total unmatched roles for LLM:      {len(unmatched_roles)}")

# --- Optional: Save for review or backup ---
pd.DataFrame({'Role': unmatched_roles}).to_csv(
    '/content/drive/MyDrive/knowledge-graph-llms/unmatched_roles_cleaned_graph.csv',
    index=False
)
print("Saved unmatched role list for LLM review.")


Graph loaded. Total nodes: 2429
Total unique :Role nodes in graph: 133
Total roles matched with OSCA:     53
Total unmatched roles for LLM:      80
Saved unmatched role list for LLM review.


In [49]:
import pandas as pd
from openai import OpenAI
import csv
import io

# 1. Load roles from the saved unmatched role list
roles_path = '/content/drive/MyDrive/knowledge-graph-llms/unmatched_roles_cleaned_graph.csv'
roles = pd.read_csv(roles_path)['Role'].dropna().tolist()

# 2. Build the prompt with all roles and few-shot examples
few_shot_examples = """Source_Role|Canonical_Role
Administration Assistant|Administrative Assistant
Administrative Assistant|Administrative Assistant
Bim Coordinator|BIM Professional
Bim Manager|BIM Professional
Bim Specialist|BIM Professional
"""

role_lines = '\n'.join(r for r in roles)
prompt = (
    "You are a workforce classification and ontology expert. Your job is to group construction and built environment job titles that are synonyms, near-synonyms, or variants describing the same core function, regardless of seniority, wording, or local jargon.\n"
    "For each group of synonymous roles, assign a single clear and concise canonical role title. This canonical title should represent the core role, not seniority, location, or organizational context.\n"
    "- Treat “Coordinator” and “Administrator” as equivalent if they serve the same function.\n"
    "- Treat abbreviations, plurals, and singulars as the same (e.g., “BIM Specialist” and “BIM Specialists”).\n"
    "- Merge generic or functionally identical roles, even if worded differently (e.g., “Administration Assistant”, “Administrative Assistant” → “Administrative Assistant”).\n"
    "- If a role does not have an obvious synonym, keep it as its own canonical title.\n"
    "- Ignore differences in level (junior/senior), project/field/site, or small title variants if the underlying job is the same.\n"
    "\nFormat your response as a CSV with two columns, separated by a pipe (`|`): Source_Role|Canonical_Role\n"
    "Do not add any explanation or commentary. Only output the CSV.\n\n"
    "Here are some examples:\n"
    f"{few_shot_examples}\n"
    "Here are the roles to group:\n"
    f"{role_lines}"
)

# 3. Call the LLM (use gpt-4o if available)
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o",  # Use 'gpt-4o' or your best available
    messages=[
        {"role": "system", "content": "You are a workforce role ontology expert."},
        {"role": "user", "content": prompt}
    ],
    max_completion_tokens=2000  # Should be plenty for 80 lines of CSV output
)

llm_csv = response.choices[0].message.content.strip()

# 4. Parse output as CSV (pipe delimiter, skip any non-header lines)
reader = csv.reader(io.StringIO(llm_csv), delimiter='|')
rows = [row for row in reader if len(row) == 2 and row[0] != "Source_Role"]

df = pd.DataFrame(rows, columns=["Source_Role", "Canonical_Role"])
output_path = '/content/drive/MyDrive/knowledge-graph-llms/unmatched_roles_canonicalized.csv'
df.to_csv(output_path, index=False)
print(f"Saved canonicalization mapping to: {output_path}")

# Show sample for review
print(df.head(10))


Saved canonicalization mapping to: /content/drive/MyDrive/knowledge-graph-llms/unmatched_roles_canonicalized.csv
                       Source_Role                   Canonical_Role
0         Administration Assistant         Administrative Assistant
1         Administrative Assistant         Administrative Assistant
2   Analytics And Insights Manager   Analytics And Insights Manager
3              Analytics Developer              Analytics Developer
4  Artificial Intelligence Manager  Artificial Intelligence Manager
5                   Asset Engineer                   Asset Engineer
6               Automation Manager               Automation Manager
7      Automation Systems Engineer      Automation Systems Engineer
8                  Bim Coordinator                 BIM Professional
9                      Bim Manager                 BIM Professional


In [50]:
# ==============================================================================
# 0. SETUP: INSTALL NECESSARY LIBRARIES
# ==============================================================================
# openpyxl is required by pandas to read .xlsx files.
!pip install openpyxl --quiet
print("Setup complete. Libraries installed.")

# ==============================================================================
# 1. PREPARATION: IMPORTS AND GOOGLE DRIVE SETUP
# ==============================================================================
import pandas as pd
import networkx as nx
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

# ==============================================================================
# 2. LOAD GRAPH AND CURATED MAPPING FILE
# ==============================================================================
# --- Define file paths ---
graph_path = '/content/drive/MyDrive/knowledge-graph-llms/graph_backups/cleaned_graph_after_noise_removal.graphml'
mapping_file = '/content/drive/MyDrive/knowledge-graph-llms/analysis_without_neo4j/Curated_Canonical_Role.xlsx'
output_graph_path = '/content/drive/MyDrive/knowledge-graph-llms/graph_backups/graph_after_canonical_merge.graphml'

# --- Load graph ---
try:
    G = nx.read_graphml(graph_path)
    print(f"Graph loaded successfully. Total nodes: {G.number_of_nodes()}, Total edges: {G.number_of_edges()}.")
except FileNotFoundError:
    print(f"ERROR: Graph file not found at {graph_path}")
    # Stop execution if the graph isn't found
    exit()

# --- Load curated mapping file ---
try:
    df = pd.read_excel(mapping_file)
    print(f"Loaded {len(df)} role mapping rows from Excel file.")
except FileNotFoundError:
    print(f"ERROR: Mapping file not found at {mapping_file}")
    # Stop execution if the mapping isn't found
    exit()

# ==============================================================================
# 3. EXPERT FIX: CREATE A HIGH-PERFORMANCE LOOKUP INDEX
# This is the performance fix. Instead of searching the whole graph for each
# role, we build a dictionary once for instantaneous lookups.
# ==============================================================================
role_id_to_node_key = {
    d.get('id'): n
    for n, d in G.nodes(data=True)
    if d.get('labels') == ':Role' and d.get('id') is not None
}
print(f"Created high-performance lookup index for {len(role_id_to_node_key)} roles.")

# ==============================================================================
# 4. BUILD MERGE GROUPS
# This logic was correct and remains the same. It groups source roles by their
# target canonical role.
# ==============================================================================
merge_map = {}
for _, row in df.iterrows():
    src = row['Source_Role'].strip()
    canon = row['Curated_Canonical_Role'].strip()
    if canon not in merge_map:
        merge_map[canon] = []
    merge_map[canon].append(src)
print(f"Built {len(merge_map)} consolidation groups.")

# ==============================================================================
# 5. MERGE NODES WITH CORRECT AND ROBUST LOGIC
# This section contains the critical fixes for both performance and correctness.
# ==============================================================================
print("\nStarting node consolidation process...")
total_merged_nodes = 0

for canon_role, src_roles in merge_map.items():
    # Use the high-performance index to get node keys
    node_keys = [role_id_to_node_key.get(role_name) for role_name in src_roles if role_name in role_id_to_node_key]

    # Skip if no nodes were found (e.g., roles already merged/deleted)
    if not node_keys:
        print(f"INFO: No graph nodes found for merge group: {src_roles} -> {canon_role}. Skipping.")
        continue

    # Skip if there's only one node and it's already the canonical one.
    # This handles cases where a role is its own canonical role.
    if len(node_keys) <= 1 and src_roles[0] == canon_role:
      continue

    # The first node in the list becomes the 'survivor' that others merge into
    survivor_key = node_keys[0]
    nodes_to_merge = node_keys[1:]

    # EXPERT FIX: To prevent data loss, we must check for existing relationships
    # based not just on the connected nodes, but on the relationship's full data
    # (type, properties, etc.). We create sets of edge "fingerprints".
    survivor_out_edges = {(target, frozenset(data.items())) for _, target, data in G.out_edges(survivor_key, data=True)}
    survivor_in_edges = {(source, frozenset(data.items())) for source, _, data in G.in_edges(survivor_key, data=True)}

    # Merge all relationships from other nodes into the survivor
    for key_to_merge in nodes_to_merge:
        # Copy outgoing edges, preventing duplicates
        for _, target, data in list(G.out_edges(key_to_merge, data=True)):
            edge_fingerprint = (target, frozenset(data.items()))
            if edge_fingerprint not in survivor_out_edges:
                G.add_edge(survivor_key, target, **data)
                survivor_out_edges.add(edge_fingerprint)

        # Copy incoming edges, preventing duplicates
        for source, _, data in list(G.in_edges(key_to_merge, data=True)):
            edge_fingerprint = (source, frozenset(data.items()))
            if edge_fingerprint not in survivor_in_edges:
                G.add_edge(source, survivor_key, **data)
                survivor_in_edges.add(edge_fingerprint)

        # Remove the now-merged node
        G.remove_node(key_to_merge)
        total_merged_nodes += 1

    # Finally, update the survivor's 'id' to the canonical role name
    G.nodes[survivor_key]['id'] = canon_role
    print(f"SUCCESS: Merged {len(nodes_to_merge)} role(s) into '{canon_role}'.")

print(f"\nConsolidation complete. Total nodes merged away: {total_merged_nodes}.")

# ==============================================================================
# 6. SAVE UPDATED GRAPH
# ==============================================================================
nx.write_graphml(G, output_graph_path)
print(f"\nSaved updated graph after consolidation to: {output_graph_path}")

# ==============================================================================
# 7. FINAL VERIFICATION
# ==============================================================================
# Check: Print all remaining Role nodes to confirm the consolidation
role_nodes = sorted([d['id'] for _, d in G.nodes(data=True) if d.get('labels') == ':Role'])
print(f"\nVerification complete. Total remaining unique :Role nodes: {len(role_nodes)}")
print("Sample of final canonical roles:", role_nodes[:20])

Setup complete. Libraries installed.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Graph loaded successfully. Total nodes: 2429, Total edges: 2883.
Loaded 80 role mapping rows from Excel file.
Created high-performance lookup index for 133 roles.
Built 56 consolidation groups.

Starting node consolidation process...
SUCCESS: Merged 2 role(s) into 'Administrative Assistant'.
SUCCESS: Merged 2 role(s) into 'Analytics And Insights Manager'.
SUCCESS: Merged 1 role(s) into 'Asset Engineer'.
SUCCESS: Merged 2 role(s) into 'Automation Systems Engineer'.
SUCCESS: Merged 2 role(s) into 'BIM Professional'.
SUCCESS: Merged 6 role(s) into 'Project Engineer'.
SUCCESS: Merged 1 role(s) into 'Human Resources And Safety Manager'.
SUCCESS: Merged 2 role(s) into 'Safety Advisor'.
SUCCESS: Merged 0 role(s) into 'IOT Software Engineer'.
SUCCESS: Merged 0 role(s) into 'ICT Support Team Leade

In [51]:
# ==============================================================================
# 0. SETUP: INSTALL NECESSARY LIBRARIES
# ==============================================================================
# No new libraries are needed if you are running this in the same session.
# This is here for completeness if you run it in a new notebook.
!pip install pandas networkx openpyxl --quiet
print("Setup complete.")

# ==============================================================================
# 1. PREPARATION: IMPORTS AND GOOGLE DRIVE SETUP
# ==============================================================================
import pandas as pd
import networkx as nx
from collections import Counter
from google.colab import drive

# Mount Google Drive to access files
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

# ==============================================================================
# 2. LOAD THE FINAL CONSOLIDATED GRAPH
# ==============================================================================
# --- Define the final graph file path ---
final_graph_path = '/content/drive/MyDrive/knowledge-graph-llms/graph_backups/graph_after_canonical_merge.graphml'

# --- Load graph with error handling ---
try:
    G = nx.read_graphml(final_graph_path)
    print(f"Final consolidated graph loaded successfully from: {final_graph_path}")
except FileNotFoundError:
    print(f"ERROR: Graph file not found at {final_graph_path}")
    # Stop execution if the graph isn't found
    exit()

# ==============================================================================
# 3. GENERATE THE GRAPH ASSESSMENT REPORT
# ==============================================================================
print("\n" + "="*50)
print("KNOWLEDGE GRAPH ASSESSMENT REPORT")
print("="*50)

# --- Overall Statistics ---
print("\n--- I. Overall Graph Statistics ---")
print(f"Total Number of Nodes: {G.number_of_nodes()}")
print(f"Total Number of Edges (Relationships): {G.number_of_edges()}")

# --- Node Type Breakdown ---
print("\n--- II. Node Type Analysis (Labels) ---")
# Use collections.Counter for an efficient way to count hashable items
node_labels = [d.get('labels', 'Unknown') for _, d in G.nodes(data=True)]
node_counts = Counter(node_labels)

print(f"{'Node Label':<25} | {'Count'}")
print("-" * 40)
for label, count in sorted(node_counts.items()):
    print(f"{label:<25} | {count}")

# --- Relationship Type Breakdown ---
print("\n--- III. Relationship Type Analysis (Labels) ---")
edge_labels = [d.get('label', 'Unknown') for _, _, d in G.edges(data=True)]
edge_counts = Counter(edge_labels)

print(f"{'Relationship Label':<25} | {'Count'}")
print("-" * 40)
for label, count in sorted(edge_counts.items()):
    print(f"{label:<25} | {count}")


# --- Specific Sanity Checks ---
print("\n--- IV. Specific Role Count Verification ---")
# Verify the final number of canonical roles
role_count = node_counts.get(':Role', 0)
print(f"Canonical Project Roles (:Role): {role_count}")
if role_count == 109:
    print("  -> VERIFIED: Count matches the expected outcome of consolidation.")
else:
    print(f"  -> WARNING: Count ({role_count}) does not match the expected 109.")

# Verify the number of OSCA roles
osca_role_count = node_counts.get(':OSCAOccupation', 0)
print(f"Standard OSCA Roles (:OSCAOccupation): {osca_role_count}")
if osca_role_count == 43:
    print("  -> VERIFIED: Count matches the expected number of unique OSCA roles.")
else:
    print(f"  -> WARNING: Count ({osca_role_count}) does not match the expected 43.")

print("\n" + "="*50)
print("End of Report")
print("="*50)



Setup complete.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
Final consolidated graph loaded successfully from: /content/drive/MyDrive/knowledge-graph-llms/graph_backups/graph_after_canonical_merge.graphml

KNOWLEDGE GRAPH ASSESSMENT REPORT

--- I. Overall Graph Statistics ---
Total Number of Nodes: 2405
Total Number of Edges (Relationships): 2829

--- II. Node Type Analysis (Labels) ---
Node Label                | Count
----------------------------------------
:OSCAOccupation           | 43
:Role                     | 109
:Skill                    | 2
:Soft skill               | 198
:Task                     | 824
:Technical skill          | 912
:Tool                     | 317

--- III. Relationship Type Analysis (Labels) ---
Relationship Label        | Count
----------------------------------------
INVOLVES_SKILL            | 488
MATCHED_TO_OSCA           | 53
PERFOR